In [3]:
import csv
import os
import glob
import pandas as pd
import numpy as np

from pandas import read_csv
from importlib.machinery import SourceFileLoader

path_wcpd = '/Users/gd/GitHub/WorldCarbonPricingDatabase/_dataset/data' 
path_dependencies = '/Users/gd/GitHub/ECP/_code/compilation/dependencies'

ecp_general = SourceFileLoader('general', path_dependencies+'/ecp_v3_gen_func.py').load_module()
ecp_cov_fac = SourceFileLoader('coverage_factors', path_dependencies+'/ecp_v3_coverage_factors.py').load_module()

gases = ["CO2"] #CH4, N2O, F-GASES #CO2 only for now

Loading data

In [9]:
for gas in gases:

    # LOAD WCPD DATAFRAMES

    wcpd_ctry = ecp_general.concatenate(path_wcpd+"/"+gas+"/national")
    wcpd_subnat = ecp_general.concatenate(path_wcpd+"/"+gas+"/subnational")
    wcpd_all = pd.concat([wcpd_ctry, wcpd_subnat]).sort_values(by=["jurisdiction", "year"])

    # ADD COLUMN WITH IEA SECTOR CODES
    ipcc_iea_map = pd.read_csv("/Users/gd/GitHub/WorldCarbonPricingDatabase/_raw/_aux_files/IPCC2006-IEA-category-codes.csv", 
                    usecols=["IPCC_CODE", "IEA_CODE"])
    ipcc_iea_map.columns = ["ipcc_code", "iea_code"]

    wcpd_all = wcpd_all.merge(ipcc_iea_map, on=["ipcc_code"], how="left")

    # LISTS OF JURISDICTION NAMES

    ctry_names = list(wcpd_ctry.jurisdiction.unique())
    subnat_names = list(wcpd_subnat.jurisdiction.unique())

    std_ctry_names = [x.replace(".", "").replace(",", "").replace(" ", "_") for x in ctry_names]
    countries_dic = dict(zip(ctry_names, std_ctry_names))

    std_subnat_names = [x.replace(".", "").replace(",", "").replace(" ", "_") for x in subnat_names]
    subnat_dic = dict(zip(subnat_names, std_subnat_names))

    # ADD COVERAGE FACTORS 

    wcpd_all = ecp_cov_fac.coverage_factors(wcpd_all, gas)

In [12]:
combustion_ipcc = ["1A1A1", "1A1A2", "1A1A3", "1A1B", "1A2A", "1A2B", "1A2C",
                   "1A2D", "1A2E", "1A2F", "1A2G", "1A2H", "1A2I", "1A2J", "1A2K", "1A2L",
                   "1A2M", "1A3A1", "1A3A2", "1A3B", "1A3C", "1A3D1", "1A3D2", "1A3E1",
                   "1A4A", "1A4B", "1A4C", "1A4C1", "1A4C2", "1A4C3", "1A5A", "1A5B", "1A5C"]
# "1A1C" - excluding one of the two IPCC categories associated with IEA flow ABFLOW011

def cf_weighted_prices(gas):

    global prices_usd, prices_usd_comb

    # PRICES
    prices_usd = ecp_general.concatenate("/Users/gd/GitHub/ECP/_raw/wcpd_usd/"+gas)

    # currently including the price of the main tax or ets scheme; should be revised to account for all schemes
    prices_usd = prices_usd[["jurisdiction", "year", "ipcc_code", "iea_code", "Product", "tax_rate_incl_ex_kusd", "ets_price_kusd"]]

    prices_usd = prices_usd.merge(wcpd_all[["jurisdiction", "year", "ipcc_code", "iea_code", "Product", "tax_cf", "ets_cf"]], 
                                on=["jurisdiction", "year", "ipcc_code", "iea_code", "Product"])

    prices_usd["tax_rate_incl_ex_kusd"] = prices_usd.tax_rate_incl_ex_kusd*prices_usd.tax_cf
    prices_usd["ets_price_kusd"] = prices_usd.ets_price_kusd*prices_usd.ets_cf

    prices_usd.drop(["tax_cf", "ets_cf"], axis=1, inplace=True)

    # calculate total price by summing across all mechanisms columns
    price_columns = [x for x in prices_usd.columns if x.startswith("tax") or x.startswith("ets")]
    prices_usd["all_inst_kusd"] = prices_usd[price_columns].sum(axis=1)

    prices_usd  = prices_usd[["jurisdiction", "year", "ipcc_code", "iea_code", "Product", "tax_rate_incl_ex_kusd", "ets_price_kusd", "all_inst_kusd"]].sort_values(by=["jurisdiction", "year"])
    prices_usd_comb = prices_usd[prices_usd["ipcc_code"].isin(combustion_ipcc)]

    #prices_usd_comb.fillna("NA", inplace=True)

    #for jur in countries_dic.keys():
    #    prices_usd.loc[prices_usd.jurisdiction==jur, :].to_csv("/Users/gd/GitHub/ECP/_raw/wcpd_weighted_prices_usd/prices_usd_"+gas+"_"+countries_dic[jur]+".csv", index=None)
    #for jur in subnat_dic.keys():
    #    prices_usd.loc[prices_usd.jurisdiction==jur, :].to_csv("/Users/gd/GitHub/ECP/_raw/wcpd_weighted_prices_usd/prices_usd_"+gas+"_"+subnat_dic[jur]+".csv", index=None)



Calculating sector-level emissions shares

In [22]:
df_sec_price = {}

for gas in gases:
    cf_weighted_prices(gas)

    # EMISSIONS
    inventory_nat = pd.read_csv("/Users/gd/OneDrive - rff/Documents/Research/projects/ecp/ecp_dataset/source_data/ghg_inventory/processed/inventory_nat_"+gas+".csv")
    inventory_nat_comb = inventory_nat[['jurisdiction', 'year', 'ipcc_code', 'iea_code', 'Product', gas]].loc[inventory_nat.ipcc_code.isin(combustion_ipcc), :]

    #recalculating ipcc category level totals (to account for rounding errors)
    inventory_nat_comb_sectot = inventory_nat_comb.groupby(by=["jurisdiction", "year", "iea_code"]).sum()
    inventory_nat_comb_sectot.reset_index(inplace=True)
    inventory_nat_comb_sectot.rename(columns={gas:gas+"_sectot"}, inplace=True)

    inventory_nat_comb = inventory_nat_comb.merge(inventory_nat_comb_sectot, on=["jurisdiction", "year", "iea_code"])
    inventory_nat_comb[gas+"_sharesec"] = inventory_nat_comb[gas]/inventory_nat_comb[gas+"_sectot"]

    inventory_nat_comb.loc[:, gas+"_sharesec"] = inventory_nat_comb.loc[:, gas+"_sharesec"].fillna(0)
    inventory_nat_comb = inventory_nat_comb.drop([gas, gas+"_sectot"], axis=1)

    # extending inventory to years beyond the last year
    for yr in range(2019, 2020+1):
        temp = inventory_nat_comb.loc[inventory_nat_comb.year==2018, :].copy()
        temp["year"].replace(to_replace={2018:yr}, inplace=True)

        inventory_nat_comb = pd.concat([inventory_nat_comb, temp])

    sec_price_comb = inventory_nat_comb.merge(prices_usd_comb, on=["jurisdiction", "year", "ipcc_code", "iea_code", "Product"], how='left')

    # Time-varying weights
    # Combustion categories
    #The summation will not work with NA values
    sec_price_comb["ets_price_kusd"].fillna(0, inplace=True)
    sec_price_comb["tax_rate_incl_ex_kusd"].fillna(0, inplace=True)

    sec_price_comb.loc[:, "ecp_ets_kusd"] = (sec_price_comb.loc[:, "ets_price_kusd"])*sec_price_comb.loc[:, gas+"_sharesec"]
    sec_price_comb.loc[:, "ecp_tax_kusd"] = (sec_price_comb.loc[:, "tax_rate_incl_ex_kusd"])*sec_price_comb.loc[:, gas+"_sharesec"]
    sec_price_comb.loc[:, "ecp_all_kusd"] = (sec_price_comb.loc[:, "ecp_ets_kusd"] + sec_price_comb.loc[:, "ecp_tax_kusd"])

    sec_price_comb = sec_price_comb.drop(["all_inst_kusd", "ets_price_kusd", "tax_rate_incl_ex_kusd"], axis=1)

    sec_price_sum = sec_price_comb.groupby(["jurisdiction", "year", "ipcc_code"]).sum()
    sec_price_sum.reset_index(inplace=True)

    iea_codes = prices_usd[["jurisdiction", "year", "ipcc_code", "iea_code"]]
    iea_codes.drop_duplicates(inplace=True)

    sec_price_sum = sec_price_sum.merge(iea_codes, on=["jurisdiction", "year", "ipcc_code"], how='left')

    # Non combustion sectors
    df_sec_price[gas] = prices_usd.loc[~prices_usd.ipcc_code.isin(combustion_ipcc)]
    df_sec_price[gas] = df_sec_price[gas].loc[df_sec_price[gas].jurisdiction.isin(inventory_nat_comb.jurisdiction.unique())]
    df_sec_price[gas].drop(["Product"], axis=1, inplace=True)
    df_sec_price[gas].drop_duplicates(["jurisdiction", "year", "ipcc_code"], inplace=True)

    df_sec_price[gas]["ets_price_kusd"].fillna(0, inplace=True)
    df_sec_price[gas]["tax_rate_incl_ex_kusd"].fillna(0, inplace=True)

    df_sec_price[gas].rename(columns={'tax_rate_incl_ex_kusd':"ecp_tax_kusd", 'ets_price_kusd':"ecp_ets_kusd"},
                        inplace=True)
    df_sec_price[gas].loc[:, "ecp_all_kusd"] = df_sec_price[gas].loc[:, "ecp_tax_kusd"] + df_sec_price[gas].loc[:, "ecp_ets_kusd"]

    df_sec_price[gas] = pd.concat([df_sec_price[gas], sec_price_sum])
    df_sec_price[gas].sort_values(by=["jurisdiction", "year", "ipcc_code"], inplace=True)

    df_sec_price[gas] = df_sec_price[gas].drop(["all_inst_kusd"], axis=1)
    df_sec_price[gas] = df_sec_price[gas].fillna(value="NA")

    df_sec_price.to_csv("/Users/gd/GitHub/ECP/_dataset/price/ecp_sectors/ecp_sector_"+gas+".csv", index=None)


/var/folders/hs/0dcrvt2s3gj57xgyfxrq3ft80000gn/T/ipykernel_8370/3809002078.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iea_codes.drop_duplicates(inplace=True)


Constant, jurisdiction-specific, weights [TO BE UPDATED]

In [ ]:

# sector-level, recording year prior to first year of pricing mechanism implementation

first_year_sector = first_year.groupby(["Jurisdiction", "Year", "IPCC_cat_code", "IEA_CODE"]).sum()
first_year_sector.loc[:, "pricing"] = np.where(first_year_sector.loc[:, "pricing"] > 0, 1.0, 0.0)
first_year_sector.reset_index(inplace=True)
first_year_sector = first_year_sector.drop_duplicates(subset=["Jurisdiction", "IEA_CODE"]) #"IPCC_cat_code" - has to be sorted based on IEA_CODE because there are two IPCC catefories corresponding to IEA ABFLOW011
first_year_sector.loc[:, "Year"] = first_year_sector.loc[:, "Year"]-1 # to take the year before first year of implementation
first_year_sector = first_year_sector.drop("pricing", axis=1)

# same adjustment
first_year_sector.loc[(first_year_sector.Jurisdiction=="Finland") & (first_year_sector.Year==1989), "Year"] = 1990
first_year_sector.loc[(first_year_sector.Jurisdiction=="Poland") & (first_year_sector.Year==1989), "Year"] = 1990


In [395]:
def ecp_const_intro_sec(share_df, temp_cp, prices):
    
    df_concat = pd.DataFrame()

    for jur in share_df.Jurisdiction.unique():
        sectors = first_year_sector.loc[(first_year_sector.Jurisdiction==jur), :]["IEA_CODE"].unique()
        
        for sector in share_df.loc[share_df.Jurisdiction==jur, "Flow"].unique():
            if jur in first_year_sector.Jurisdiction.unique() and sector in sectors:
                weight_year = first_year_sector.loc[(first_year_sector.Jurisdiction==jur) & (first_year_sector.IEA_CODE==sector)]["Year"].item()
            else:
                weight_year = 2015
    
            temp_df = share_df[(share_df["Jurisdiction"]==jur) & (share_df["Year"]==weight_year) & (share_df["Flow"]==sector)]
            temp_df.drop("Year", axis=1, inplace=True)

            merge_keys = ["Jurisdiction", "Flow", "Product"]
            temp_df = temp_df.merge(prices, on=merge_keys, how="left")

            temp_df["ETS_ew_price_sector_2019USD"] = (temp_df.ETS_price_2019USD)*temp_df.CO2_em_sharesec
            temp_df["Tax_ew_price_sector_2019USD"] = (temp_df.Tax_rate_incl_ex_2019USD)*temp_df.CO2_em_sharesec
            temp_df["Total_ew_price_sector_2019USD"] = (temp_df.Tax_rate_incl_ex_2019USD + temp_df.ETS_price_2019USD)*temp_df.CO2_em_sharesec

            temp_df.drop(["ETS_price_2019USD", "Tax_rate_incl_ex_2019USD"], axis=1, inplace=True)

            temp_df_sum = temp_df.groupby(["Jurisdiction", "Year", "Flow"]).sum()
            temp_df_sum.reset_index(inplace=True)

            temp_df_sum = temp_df_sum[["Jurisdiction", "Year", "Flow", "ETS_ew_price_sector_2019USD", "Tax_ew_price_sector_2019USD", 
                                       "Total_ew_price_sector_2019USD"]]

            if df_concat.empty == True:
                df_concat = temp_df_sum
            else:
                df_concat = pd.concat([df_concat, temp_df_sum])
        
    return df_concat

In [ ]:
ecp_sector_intro = ecp_const_intro_sec(combustion_nat_ii, cp_all, prices_usd_sector)
ecp_sector_intro.to_csv(file_root_III+"/ecp_calculation/ecp/ecp_sectors/ecp_intro/ecp_sector_intro.csv", index=None)

Aggregate 'Buildings' and 'Elec&Heat' sectors

1. Get sector-level CO2 emissions dataframe
2. Get emissions from buildings and Elec&Heat
3. Get emissions from their respective sub-sectors
4. Calculate share of emissions of each sub-sector in aggregate sector
5. Multiply sector-level price by share
6. Aggregate for these two sectors

In [398]:
sec_price_intro = pd.read_csv(file_root_III+"/ecp_calculation/ecp/ecp_sectors/ecp_intro/ecp_sector_intro.csv")
sec_price_intro.rename(columns={"Flow":"IEA_CODE"}, inplace=True)

elec_heat = combustion_nat_ii.loc[combustion_nat_ii.Flow.isin(["ABFLOW003", "ABFLOW004", "ABFLOW005"]), ["Jurisdiction", "Year", "Flow", "Product", "CO2_Emissions"]]
build = combustion_nat_ii.loc[combustion_nat_ii.Flow.isin(["ABFLOW034", "ABFLOW035"]), ["Jurisdiction", "Year", "Flow", "Product", "CO2_Emissions"]]

elec_heat = elec_heat.groupby(["Jurisdiction", "Year", "Flow"]).sum()
elec_heat.reset_index(inplace=True)
build = build.groupby(["Jurisdiction", "Year", "Flow"]).sum()
build.reset_index(inplace=True)

agg_eh = elec_heat.groupby(["Jurisdiction", "Year"]).sum()
agg_eh.reset_index(inplace=True)
agg_build = build.groupby(["Jurisdiction", "Year"]).sum()
agg_build.reset_index(inplace=True)

elec_heat = elec_heat.merge(agg_eh, on=["Jurisdiction", "Year"])
build = build.merge(agg_build, on=["Jurisdiction", "Year"])

elec_heat["CO2_Emissions_sharesec"] = elec_heat.CO2_Emissions_x / elec_heat.CO2_Emissions_y
build["CO2_Emissions_sharesec"] = build.CO2_Emissions_x / build.CO2_Emissions_y
                                                          

In [399]:
df_concat = pd.DataFrame()

share_df = build

def agg_sec(share_df, flow_name):
    
    df_concat = pd.DataFrame()
    
    for jur in share_df.Jurisdiction.unique():
        sectors = first_year_sector.loc[(first_year_sector.Jurisdiction==jur), :]["IEA_CODE"].unique()

        for sector in share_df.loc[share_df.Jurisdiction==jur, "Flow"].unique():
            if jur in first_year_sector.Jurisdiction.unique() and sector in sectors:
                weight_year = first_year_sector.loc[(first_year_sector.Jurisdiction==jur) & (first_year_sector.IEA_CODE==sector)]["Year"].item()
            else:
                weight_year = 2015

            temp_df = share_df[(share_df["Jurisdiction"]==jur) & (share_df["Year"]==weight_year) & (share_df["Flow"]==sector)]
            temp_df.drop("Year", axis=1, inplace=True)

            temp_prices = sec_price_intro.loc[(sec_price_intro["Jurisdiction"]==jur) & (sec_price_intro["IEA_CODE"]==sector), :]

            temp_df = temp_df.merge(temp_prices, left_on=["Jurisdiction", "Flow"], right_on=["Jurisdiction", "IEA_CODE"], how="left")

            temp_df.loc[:, "ETS_ew_price_sector_2019USD_sec"] = (temp_df.loc[:, "ETS_ew_price_sector_2019USD"])*temp_df.loc[:, "CO2_Emissions_sharesec"]
            temp_df.loc[:, "Tax_ew_price_sector_2019USD_sec"] = (temp_df.loc[:, "Tax_ew_price_sector_2019USD"])*temp_df.loc[:, "CO2_Emissions_sharesec"]
            temp_df.loc[:, "Total_ew_price_sector_2019USD_sec"] = (temp_df.loc[:, "Total_ew_price_sector_2019USD"])*temp_df.loc[:, "CO2_Emissions_sharesec"]

            if df_concat.empty == True:
                df_concat = temp_df
            else:
                df_concat = pd.concat([df_concat, temp_df])

    df_concat = df_concat[["Jurisdiction", "Year", "IEA_CODE", "ETS_ew_price_sector_2019USD_sec", "Tax_ew_price_sector_2019USD_sec", "Total_ew_price_sector_2019USD_sec"]]
    df_concat = df_concat.groupby(["Jurisdiction", "Year"]).sum()

    df_concat["Flow"] = flow_name
    
    return df_concat
    
#Merge with sector-level price dataframe        

In [400]:
ecp_intro_buildings = agg_sec(build, "buildings")
ecp_intro_elecheat = agg_sec(elec_heat, "ABFLOW002")

ecp_intro_buildings.to_csv(file_root_III+"/ecp_calculation/ecp/ecp_sectors/ecp_intro/ecp_buildings_intro.csv")
ecp_intro_elecheat.to_csv(file_root_III+"/ecp_calculation/ecp/ecp_sectors/ecp_intro/ecp_elecheat_intro.csv")